# Loss functions

1、分类任务 损失函数 
  - 二分类 ： 二分类交叉熵损失函数  Binary Cross-Entropy 
  - 多分类 ： 多分类交叉熵损失函数  Categorical Cross-Entropy 
  - Focal Loss ： 优化版的交叉熵损失函数， 用于解决难\易分类样本数量不均衡的问题  

2、回归任务 损失函数
  - 均方误差 MSE （Mean Squared Error）
  - 平均绝对误差 MAE （Mean Absolute Error）

3、目标检测任务 损失函数

目标检测任务的损失函数由 类别损失（Classificition Loss） 和 定位损失（Bounding Box Regeression Loss） 两部分构成。
- 类别损失（分类损失） ： 
  - 交叉熵损失 
  - Focal Loss
- 定位损失 (回归损失)：
  - SmoothL1 Loss   
  - IoU Loss  
  - GIoU Loss 
  - DIoU Loss 
  - CIoU Loss 
  - EIoU Loss 
  - Alpha IoU Loss 
  - SIoU Loss 
  - WIoU Loss 
  - Shape-IoU  Loss 

## 二元交叉熵损失函数 Binary Cross Entropy Loss

1、二元交叉熵损失函数

（1）二元交叉熵损失函数 （Binary Cross Entropy Loss） 适用于二分类问题 ： 样本标签为二元值：0 或 1

（2）用于将 模型 预测值 和 真实标签值 之间的差异转化为一个标量值，从而衡量模型预测的准确性。
          计算公式 ：

$$L = -\frac{1}{N} \sum_{i=1}^{N} [y_i \log(\hat{y_i}) + (1-y_i) \log(1-\hat{y_i})]$$
其中：
- $N$ 表示样本数量
- $y_i$ 表示第 $i$个样本的真实标签
- $\hat{y_i}$ 表示第 $i$个样本的预测值

如果 
$y_i=1$，则第一项 $y_i \log(\hat{y_i})$ 生效，第二项 $(1-y_i) \log(1-\hat{y_i})$ 失效

如果 $y_i=0$，则第一项 $y_i \log(\hat{y_i})$ 失效，第二项 $(1-y_i) \log(1-\hat{y_i})$ 生效

### 2、 nn.BCELoss() 类

`nn.BCELoss()` 是pytorch 实现的二元交叉熵损失函数，也称为对数损失函数（Log Loss）

```python
torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
```

参数说明：
  - weight ：用于样本加权的权重张量。如果给定，则必须是一维张量，大小等于输入张量的大小。默认值为 None。
  - reduction ：指定如何计算损失值。可选值为 'none'、'mean' 或 'sum'。默认值为 'mean'

### 3、使用场景举例

假设有一个 二分类任务：判断图片中是否包含猫。

该图像的标签值为  0 或 1

我们可以定义一个二元分类模型，用 Sigmoid 输出一个概率值，表示样本属于猫的概率。

```python
import torch
import torch.nn as nn

class CatClassifier(nn.Module):
    def __init__(self):
        super(CatClassifier, self).__init__()
        self.fc = nn.Linear(5, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x


model = CatClassifier()
criterion = nn.BCELoss()

x = torch.rand((3, 5))
label = torch.tensor([0, 1, 1], dtype=torch.float32)
pred = model(x)  # tensor([[0.6140],[0.5350],[0.5852]], grad_fn=<SigmoidBackward0>)
loss = criterion(pred.squeeze(), label)
print(loss)
```

### 4、torch.nn.BCEWithLogitsLoss()  与 nn.BCELoss() 的区别

`nn.BCELoss()`  的输入是 二元分类模型的预测值 $\hat{y}$ 和 实际标签 $y$。并且$\hat{y}$的范围是 [0,1]，因为二元分类模型内部已经对预测结果做了 sigmoid 处理。

$$nn.BCELoss()=-\frac{1}{N}\sum_{i=1}^N[y_i\log(\hat{y_i})+(1-y_i)\log(1-\hat{y_i})]$$

`torch.nn.BCEWithLogitsLoss()` 的输入也是 二元分类模型的输出值 $z$ 和实际标签 $y$，不同的是输出$z$在模型内部没有经过 sigmoid 处理，是任意实数。  这种情况下，sigmoid 处理就被放到了损失函数中，

所以，`torch.nn.BCEWithLogitsLoss()`  函数内部的计算过程是先对 $z$ 应用 sigmoid 函数，将其映射到 [0,1] 范围内，然后再使用二元交叉熵 计算预测值和实际标签之间的损失值。

$$nn.BCEWithLogitsLoss()=-\frac{1}{N}\sum_{i=1}^N[y_i\log\sigma(z_i)+(1-y_i)\log(1-\sigma(z_i))]$$

另外，`torch.nn.BCEWithLogitsLoss() `还支持设置 pos_weight 参数，用于处理样本不平衡的问题。而 `nn.BCELoss()` 不支持设置 pos_weight 参数。

### 5、torch.nn.BCEWithLogitsLoss() 类

```python
torch.nn.BCEWithLogitsLoss(weight=None,
                           size_average=None,                            
                           reduce=None, 
                           reduction='mean',     
                           pos_weight=None)
```

参数：
  - weight：用于对每个样本的损失值进行加权。默认值为 None。
  - reduction：指定如何对每个 batch 的损失值进行降维。可选值为 ‘none’、‘mean’ 和 ‘sum’。默认值为 ‘mean’。
  - pos_weight：用于对正样本的损失值进行加权。可以用于处理样本不平衡的问题。例如，如果正样本比负样本少很多，可以设置 pos_weight 为一个较大的值，以提高正样本的权重。默认值为 None。

## 交叉熵损失函数 Cross Entropy Loss

### 1、交叉熵损失函数

交叉熵损失多用于 多分类任务，下面我们通过拆解交叉熵的公式来理解其作为损失函数的意义

假设我们在做一个  n分类的问题，模型预测的输出结果是 $[x_1,  x_2, x_3, ...., x_n]$  
然后，我们选择交叉熵损失函数作为目标函数，通过反向传播调整模型的权重

交叉熵损失函数的公式 ：

$$
\begin{aligned}
loss(x,class) & =-log(\frac{e^{x_{[class]}}}{\sum_je^{x_j}}) \\
 & =-x_{[class]}+log(\sum_je^{x_j})
\end{aligned}
$$

- $x$是预测结果，是一个向量 $x=[x_1, x_2, x_3, ...., x_n]$，其元素个数 和 类别数一样多
- class 表示这个样本的实际标签，比如，样本实际属于分类 2，那么 $class=2$  ， $x_{[class]}$ 就是$x_2$，就是取预测结果向量中的第二个元素，即，取其真实分类对应的那个类别的预测值

1. 首先，交叉熵损失函数中包含了一个最基础的部分：$softmax(x_i)=\frac{e^{x_i}}{\sum_{j=0}^ne^{x_j}}$
    softmax 将分类的结果做了归一化：
    - 先经过 $e^{x}$ 的运算，将 $x$ 转换为非负数
    - 再通过公式 $\frac{e^{x_i}}{\sum_{j=0}^ne^{x_j}}$ 计算出该样本被分到分类$i$概率，这里所有分类概率相加的总和等于1
2. 我们想要使预测结果中，真实分类的那个概率接近 100%。 我们取出真实分类的那个概率：
  $\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}$，我们希望它的值是 100%
3.  作为损失函数，后面需要参与求导。乘\除法 表达式求导比较麻烦，所以最好想办法转化成加\减法表达式。最自然的想法是取对数，把乘\除法转化为加\减法表达式：

$log{\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}} = log{e^{x_{[class]}}} - log{\sum_{j=0}^{n} e^{x_{j}}}$

  - 由于对数单调增，那么求 $\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}$ 的最大值的问题，可以转化为求 $log{\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}}$ 的最大值的问题。
  
  -  $\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}$ 的取值范围是 (0, 1)，最大值为1。 取对数之后，$log{\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}}$ 的取值范围为 [−∞,0]，最大值为0


4. 作为损失函数的意义是：当预测结果越接近真实值，损失函数的值越接近于0

  所以，我们把 $log{\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}}$ 取反之后，$-log{\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}}$  最小值为0
  
  这样就能保证当  $\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}}$ 越接近于 100%， $loss=-log(\frac{e^{x_{[class]}}}{\sum_{j=0}^{n} e^{x_{j}}})$  越接近0。

### 2、nn.CrossEntropyLoss

```python
nn.CrossEntropyLoss(weight=None, 
                    reduction='mean'，
                    ignore_index=-100)
```

参数 ：
- weight (optional): 一个张量，用于为每个类别的 loss 设置权值。可以用于处理类别不平衡的情况。
  - 默认值为None
  - weight必须是float类型的 tensor，其长度要与类别个数一致，即每一个类别都要设置权重值
  $loss(x, class) = weight_{[class]}(-log(\frac{e^{x_{[class]}}}{\sum_je^{x_{j}}}))$
- reduction (string, optional): 指定损失的计算方式，可选值有："none"、"mean"、"sum"
  - "none" ：表示不进行任何降维，返回每个样本的损失
  - "mean"：表示对参与计算的样本的损失取平均值，（ "mean" 为默认值）
  - "sum"：表示对参与计算的样本的损失求和
- ignore_index (int, optional): 忽略目标中的特定类别索引，不计入损失计算。默认值为-100

### 3、应用

假设有4张图片( batch_ size=4) 
我们需要把这4张图片分类到5个类别上去，比如说：鸟，狗，猫，汽车，船
经过网络得到的预测结果为：predict，size=[4, 5]
其真实标签为 label，size=[4]

接下来使用 `nn.CrossEntropyLoss()`计算 预测结果 predict 和 真实值label 的交叉熵损失

```python
import torch
import torch.nn as nn

# -----------------------------------------
# 定义数据: batch_size=4；  一共有5个分类
# label.size() : torch.Size([4])
# predict.size(): torch.Size([4, 5])
# -----------------------------------------
torch.manual_seed(100)
predict = torch.rand(4, 5)
label = torch.tensor([4, 3, 3, 2])
print(predict)
print(label)

# -----------------------------------------
# 直接调用函数 nn.CrossEntropyLoss() 计算 Loss
# -----------------------------------------
criterion = nn.CrossEntropyLoss()
loss = criterion(predict, label)
print(loss)
```

## 信息量、熵、交叉熵、相对熵\KL散度、交叉熵损失函数



### 1、信息量 Amount of Information

![image](/home/vtchen/00-coding/01-deep_learning/02-Loss_fun/images/1.jpg) 

### 2、熵 Entropy

![image.png](/home/vtchen/00-coding/01-deep_learning/02-Loss_fun/images/2.jpg)

### 3、交叉熵 Cross Entropy

![image.png](/home/vtchen/00-coding/01-deep_learning/02-Loss_fun/images/3.jpg)

### 4、相对熵 Relative Entropy \ KL散度  KLDivergence
![image](/home/vtchen/00-coding/01-deep_learning/02-Loss_fun/images/4.jpg)

### 5、交叉熵损失函数 Cross Entropy
![image](/home/vtchen/00-coding/01-deep_learning/02-Loss_fun/images/5.jpg)

## 均方误差损失函数 MSE

### 1、均方差损失 MSE
均方误差（ MSE ：Mean Squared Error ）损失函数，也称为平方损失函数，是用于回归任务的一种常见损失函数。
该损失函数 用于衡量 模型对于每个样本预测值与实际值之间的平方差异的平均程度。

均方误差损失函数 的数学表达式为：

$$MSE = \frac{1}{n} \sum^n_{i=1}(y_i-\hat y_i)^2$$


- 其中：
  * $n$ 是样本数量
  * $y_i$ 是第 $i$个样本的 真实标签
  * $\hat{y_i}$ 是第 $i$个样本的 预测值

- 均方误差损失函数 对差异的平方进行了求和，这使得较大的误差在计算中得到了更大的权重，这也导致它对离群值敏感，因为它会放大离群值的影响。
- 均方误差损失函数 在回归问题中广泛使用，尤其在对误差敏感度较高的情况下。

### 2、 torch.nn.MSELoss() 
```python
torch.nn.MSELoss(reduction='mean')
```
参数 reduction ：  指定损失的计算方式， 可选值有："none"、"mean"、"sum"
- reduction='None' ：按照样本原始维度输出，不做另外处理
- reduction='mean'  (默认)：对应位置求和后取平均
- reduction='sum'，：对应位置求和

### 3、应用
```python
import torch
import torch.nn as nn

# 创建一个示例的模型输出和真实标签
torch.manual_seed(121)
model_output = torch.randn(10, requires_grad=True)  # 模型的输出
true_labels = torch.randn(10)  # 真实标签

# 创建均方误差损失函数
mse_loss = nn.MSELoss()

# 计算损失
loss = mse_loss(model_output, true_labels)

# 打印损失值
print("Mean Squared Error Loss:", loss.item())
```

## 平均绝对误差损失函数 MAE

### 1、平均绝对误差损失函数 MAE 
平均绝对误差损失函数（MAE ：Mean Absolute Error），也称为 绝对值损失函数，通常用于回归任务

该损失函数 用于衡量 模型对于每个样本预测值与实际值之间的平方差异的平均程度。

平均绝对误差 损失函数的数学表达式为：
$$MAE = \frac{1}{n} \sum^n_{i=1}｜y_i-\hat y_i｜$$
其中：
  - $n$ 是样本数量
  - $y_i$ 是第 $i$个样本的 真实标签
  - $\hat{y_i}$ 是第 $i$个样本的 预测值

- 相比于均方误差 (MSE)，绝对值损失函数 (MAE) 对离群值不敏感，因为它不会对差异值求平方
- 平均绝对损失函数适用于回归任务，特别是当数据中存在离群值或对预测误差的敏感性要求较低的情况下

### 2、 torch.nn.L1Loss()
```python
torch.nn.L1Loss(reduction='mean')
```
参数 reduction ：  指定损失的计算方式， 可选值有："none"、"mean"、"sum"
- reduction='None' ：按照样本原始维度输出，不做另外处理
- reduction='mean'  (默认)：对应位置求和后取平均
- reduction='sum'，：对应位置求和

### 3、应用

```python
import torch
import torch.nn as nn

# 创建一个示例输入和目标
torch.manual_seed(121)
predictions = torch.randn((3,))  # 模型的预测值
targets = torch.randn((3,))      # 实际目标值

# 创建 MAE 损失函数实例
mae_loss = nn.L1Loss()

# 计算损失
loss = mae_loss(predictions, targets)

# 打印结果
print('MAE Loss:', loss.item())
```